In [8]:
out_path = "/home/himanshu/assignment_jupyter_notebooks/outputs/rdd/full"

In [ ]:
This file existed in the machine I used to run the code. I made an SSH connection to this machine 

In [10]:
raw_data=sc.textFile("/home/himanshu/assignment_jupyter_notebooks/small_chunk",minPartitions=88*4)

## Custom split
This function generates (word,docid,pos) tuples from the raw data. Each line in the raw data represents a document.
The first number in the line is the docid followed by the words in the document,

In [53]:
def custom_split(string):
    strspl = string.split()
    #docid=int(next(docid_counter))
    docid=strspl[0]
    positions = len(strspl)    
    pairs = [( strspl[pos],docid,pos-1 ) for pos in range(1,positions)]
    return pairs

## Custom_sorter
Sorts the list of position ids

In [54]:
def custom_sorter(element):
    #having key as (word,docid)
    element[1].sort()
    return(element[0],element[1])
    
    

##Customer_sorter2

element received in form (< word >, <[list of (docid,[list of position ids])]>)
In each element we sort the value field according to docids

In [55]:
def custom_sorter2(inv_index_element):
    word = inv_index_element[0]
    doc_pos_list = inv_index_element[1]
    doc_pos_list.sort(key=lambda x: x[0])
    return(word,doc_pos_list)

In [56]:
docs_list = raw_data.flatMap(lambda line: custom_split(line))

Using (word,docid) as key and (posid) as value. Hence now rdd elements are {(word,docid),(posid))

In [57]:
grouped_docs = docs_list.map(lambda x: ( (x[0],x[1]), x[2] ))

Using group by key so that we now have {<word,docid>, <[List of position ids]>}

In [ ]:
word_doc_pairs = grouped_docs.groupByKey().mapValues(list)

Sorting the list of position ids so that binary search is easily possible later

In [9]:
word_doc_pos_sorted = word_doc_pairs.map(lambda x: custom_sorter(x))

Converting sructure of rdd element

{(word), ( docid ,  [list_of_posid] ) }

In [10]:
word_table = word_doc_pos_sorted.map(lambda x: (  x[0][0], (x[0][1],x[1])))

Using group by key so that now we have { (word), [list of (docid,[posisd] )]}

In [11]:
inverted_index = word_table.groupByKey().mapValues(list)

Now sorting the document ids
Finally we have dictionary of format:


    key-> word
    value-> list of document_ids followed by list of position_ids in each document

In [12]:
inverted_index_sorted = inverted_index.map(lambda x: custom_sorter2(x))

Saving this unfinished inverted index for further processing

In [ ]:
inverted_index_sorted.saveAsTextFile(out_path)